# Entanglement based QKD

**Learning Objectives:**
1. Students will apply the principle of quantum entanglement and Bell's inequality to construct a quantum key distribution protocol.
1. Students will be able to use Bell's inequality to check for eavesdropping.





## 1 Introduction

Another interesting turn in the field of quantum cryptography was the development of QKD schemes based on quantum entanglement. In fact, quantum entanglement now forms the basis of new types of schemes that are coming up called measurement devices independent QKD.

Before we describe Ekert protocol (popularly called E91), the first QKD protocol using quantum entanglement, we will discuss a simpler version of the protocol.

## 2 EPR Based QKD Protocol

Consider a Bell pair $\frac{\rvert00\rangle + \rvert11\rangle}{\sqrt{2}}$. We can also work with other Bell pairs for example $\frac{\rvert01\rangle + \rvert10\rangle}{\sqrt{2}}$. However, using $\frac{\rvert01\rangle + \rvert10\rangle}{\sqrt{2}}$ will result in Alice and Bob having opposite bit strings. 
For simplicity, we can assume that these Bell pairs were created sometime in the past and now Alice and Bob each have one qubit out of it. Alternatively, we can assume that Alice and Bob are provided a qubit out of the Bell pair by some third-party source who generates such pairs. 

Suppose Alice and Bob want to exchange a secret key. When they are ready they will measure their respective qubits. The order of which the measurements take place (Alice measures first or Bob measures first) does not matter. 

To prevent a third party from obtaining the same random string Alice and Bob must measure their qubits randomly in one of the two random (non-orthogonal) bases . 



### The Protocol Flow

Assume that the bases are &#10010; and &#x2716; as before.

The protocol proceeds shown in the table below. 


|-|-|-|-|-|        
|: --- |:--:|: -- :|:--:|:--:|
| ***Alice's Random Bases*** |✚ |✖| ✖ |✚ |✖ |✖ |✚ |✚ |
| ***Alice's Observations*** |→  |↖|↖  |↑  |↖ |↗ |↑ |↑  |
| ***Bob's Random Bases***   |✖ |✖| ✚ |✚ |✖ |✚ |✖|✚  |
| ***Bob's Observations***   |↗ |↖ |↑  |↑  |↖  |→  | ↖|↑|  
| ***Bases agree***          |  | Y |  | Y  | Y  |  | |Y |

Alice and Bob keep the bits that were measured in the same basis.

To detect an eavesdropper or other sources (environment) that may have either caused a qubit to collapse or entanglement to disintegrate, Alice and Bob will publicly compare the values of some of the qubits to look for discrepancies. If any discrepancies are found, assuming an error free channel, Alice and Bob have to throw out the exchanged bits. 

![EPR Protocol Illustration](nanomod5-fig1.png)


## 3 The E91 Protocol

Ekert’s 1991 protocol is almost the same as the EPR protocol from the previous section but differs in two steps. 

First, the measurement is done in three different bases (instead of just two).

Second, the qubits that both parties measure in the same basis are saved as possible keys, the qubits that were measured in different basis are used to detect any eavesdropper, or decoherence of the entangled qubits by checking if they violate Bell's inequality (CHSH game) or not.

Of the qubits are independent qubits (not entangled) they will satisfy Bell's inequality. 


In [31]:
# Code to generate GUI for entering program parameters
import random
import ipywidgets as widgets
from IPython.display import display
from random import choices
import numpy.random as npr
import numpy

# Configuration of the widget that controls the number of qubits
number_of_qubits = widgets.IntText(
                    value=100,
                    disabled=False
                    )
qubit_control = widgets.VBox([widgets.Label(value="Enter number of initial qubits"), number_of_qubits])

# Configuration of the widget that controls whether Eve is active or not
is_Eve_active =  widgets.ToggleButton(
                    value=False,
                    description='Click to activate',
                    disabled=False,
                    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
                    tooltip='Will Eve intercept the transmission',
                    icon=''
                    )
eve_message = widgets.Label(value="Eve is not active")
eve_control = widgets.VBox([eve_message, is_Eve_active])

# Configuration of an event handler for the eve control widget
def on_value_change(change):
    is_Eve_active.description = 'Click to deactivate' if change['new'] else 'Click to activate'
    eve_message.value = 'Eve is active' if change['new'] else 'Eve is not active'
is_Eve_active.observe(on_value_change, names='value')

eve_present = is_Eve_active.value
# display the control widgets
display(qubit_control, eve_control)

In [32]:
# parameters passed into main program from GUI
n = number_of_qubits.value
eve_present = is_Eve_active.value

# Let's remind us of the parameters that we are working with
print("Program parameters have been entered into the program as follows:")
print("  * Central Source will generate {} entangled qubit pairs, sending one qubit to Alice and the other to Bob.".format(n))
print("  * Eve will {}intercept the transmission.".format('' if eve_present else 'not '))


Program parameters have been entered into the program as follows:
  * Central Source will generate 90 entangled qubit pairs, sending one qubit to Alice and the other to Bob.
  * Eve will not intercept the transmission.


In [24]:
def entangled_qubit_source(number_of_qubits):
    alice_qubits, bob_qubits = [], []
    for i in range(number_of_qubits):
        value1 = random.choice([0, 1])
        value2 = 1 - value1
        alice_qubits.append(value1)
        bob_qubits.append(value2)
    return alice_qubits, bob_qubits

def random_bases(number_of_qubits):
    alice_bases = []
    bob_bases = []
    eves_bases = []
    for i in range(number_of_qubits):
        # Generate a random choice with specified weights
        choice = choices(
            ['a2b1', 'a3b2', 'a1b1', 'a1b3', 'a3b1', 'a3b3', 'a1b2', 'a2b2', 'a2b3'],
            cum_weights=[1/6, 1/3, 4/9, 5/9, 6/9, 7/9, 23/27, 25/27, 1],
            k=1
        )[0]

        # Split the choice into Alice's and Bob's bases
        alice_base, bob_base = choice.split('b')

        # Eves bases
        eves_choice = choices(['e1', 'e2', 'e3'], cum_weights=[1/3, 2/3, 1], k=1)[0]

        # Append Alice's and Bob's bases to the lists
        alice_bases.append(alice_base)
        bob_bases.append('b' + bob_base)
        eves_bases.append(eves_choice)
    return alice_bases, bob_bases, eves_bases

def scratch_bits(alice_bases, bob_bases):
    sifted_key = []
    for alice_base, bob_base in zip(alice_bases, bob_bases):
        if alice_base == "a2" and bob_base == "b1":
            sifted_key.append(1)
        elif alice_base == "a3" and bob_base == "b2":
            sifted_key.append(1)
        else:
            sifted_key.append('W')
    return sifted_key

def sift_key(sifted_key, alice_qubits, bob_qubits):
    generated = 0
    alice_sifted_key, bob_sifted_key = [], []
    for index, value in enumerate(sifted_key):
        if value != "W":
            generated += 1
            alice_sifted_key.append(alice_qubits[index])
            flipped_value = 1 - bob_qubits[index]  # Flip the value
            bob_sifted_key.append(flipped_value)
    return alice_sifted_key, bob_sifted_key, generated

def detect_eves_pres(sifted_key, alice_bases, alice_qubits, bob_qubits, bob_bases):
    countMatrix = {
        'a1b1': {'uu': 0, 'ud': 0, 'du': 0, 'dd': 0},
        'a1b3': {'uu': 0, 'ud': 0, 'du': 0, 'dd': 0},
        'a3b1': {'uu': 0, 'ud': 0, 'du': 0, 'dd': 0},
        'a3b3': {'uu': 0, 'ud': 0, 'du': 0, 'dd': 0}
    }

    for index, value in enumerate(sifted_key):
        alice_base = alice_bases[index]
        bob_base = bob_bases[index]
        base = alice_base + bob_base

        if value == 'W' and base in countMatrix:
            alice_dir = 'u' if alice_qubits[index] == 1 else 'd'
            bob_dir = 'u' if bob_qubits[index] == 1 else 'd'
            dir = alice_dir + bob_dir

            countMatrix[base][dir] += 1

    return countMatrix

# Calculate S value from unused correlation coefficients
def calcBigOlSVal(countMatrix):
    # Initialize coefficients
    coef = {'a1b1': 0, 'a1b3': 0, 'a3b1': 0, 'a3b3': 0}

    # Calculate coefficients for each correlation
    for key in countMatrix:
        tempCount = countMatrix[key]
        totCounts = sum(tempCount.values())

        if totCounts != 0:
            coef[key] = float(tempCount['uu'] + tempCount['dd'] - tempCount['ud'] - tempCount['du']) / totCounts
        else:
            coef[key] = 0

    # Calculate and return the S value
    return coef['a1b1'] - coef['a1b3'] + coef['a3b1'] + coef['a3b3']

def printTable(caption, rows):
    """Make a nice looking table like in the book"""
    from IPython.display import HTML, display
    display(HTML(
        '<table style="border-bottom: 4px double #333;border-top: 4px double #333;padding: 10px 0;">' +
        '<caption style="font-weight:900; color:black; border-top: 4px double #333;">' + caption + '</caption>' +
        '<tr>{}</tr></table>'.format('</tr><tr>'.join(
            '<td>' + row[0] + '</td><td>{}</td>'.format('</td><td>'.join(row[1])) for row in rows)
        )
    ))


In [25]:
n = number_of_qubits.value
eve_present = is_Eve_active.value

eveStat = { 'intercepted' : 0, 'choseRight' : 0 }
eveInterceptRate = 0.9
eveDetTol = 0.9
eves_qubits = []

alice_qubits,bob_qubits = entangled_qubit_source(n)
alice_bases,bob_bases,eves_bases = random_bases(n)

if eve_present:
    for index in range(n):
        aIndex = alice_bases[index][1]
        eIndex = eves_bases[index][1]
        if npr.rand() > ( 1 - eveInterceptRate ):
            eveStat['intercepted'] += 1
            if aIndex == eIndex:
                eveStat['choseRight'] += 1
                eves_qubits.append(alice_qubits[index])
            else :
                alice_qubits[index] =  1 - alice_qubits[index]
                eves_qubits.append("N/A")
        else:
            eves_qubits.append("Not")

sifted_key = scratch_bits(alice_bases,bob_bases)
counterMatrix = detect_eves_pres(sifted_key, alice_bases, alice_qubits, bob_qubits, bob_bases)
s = calcBigOlSVal(counterMatrix)
expected = -2*numpy.sqrt(2)
ratio = numpy.abs( (s - expected ) / expected )
alice_private_key, bob_private_key,generated = sift_key(sifted_key,alice_qubits,bob_qubits)

printTable(
    caption="The number of generated digits are: " + str(generated),
    rows=[
        ["Generated&nbsp;digits", [str(generated)]],
        ["Alice's&nbsp;key", list(map(str, alice_qubits))],
        ["Bob's&nbsp;key", list(map(str, bob_qubits))],
        ["Alice's&nbsp;bases", list(map(str, alice_bases))],
        ["Bob's&nbsp;bases", list(map(str, bob_bases))],
        ["Sifted&nbsp;key", list(map(str, sifted_key))],
        ["Alice's&nbsp;private&nbsp;key", list(map(str, alice_private_key))],
        ["Bob's&nbsp;private&nbsp;key", list(map(str, bob_private_key))],
        ["Eve's&nbsp;qubits", list(map(str, eves_qubits))]])

if  ratio > eveDetTol:
    print('!!!!! Eve Detected !!!!!')
    print('An S value of {0} was found. This deviation from the true value of {3} \nis {1} times higher than the Eve tolerance of {2}%.'.format(s, round(ratio/eveDetTol, 2), 100*eveDetTol, expected ))
else:
    print('An S value of {0} was found. This is within {1}% of the expected value {2}.\nThis is below the Eve Detection tolerance of {3}% and therefore and we assume no Eve presence'.format(s, 100*round(ratio, 2), expected, 100*eveDetTol))

!!!!! Eve Detected !!!!!
An S value of 0.4143939393939394 was found. This deviation from the true value of -2.8284271247461903 
is 1.27 times higher than the Eve tolerance of 90.0%.
